In [ ]:
from __future__ import print_function
import tensorflow.python.keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Input, Reshape
from tensorflow.python.keras.layers import Conv2D, UpSampling2D, BatchNormalization
from tensorflow.python.keras.layers import LeakyReLU
from tensorflow.python.keras.layers import concatenate
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import pickle

## Load and process data

In [ ]:
with open('data.pkl', 'rb') as f:
    x_train = pickle.load(f)
x_train = x_train.astype('float32')

In [ ]:
X_train[i,:,:,j,:].shape

In [ ]:
120*144*256*12

In [ ]:
m = 4; n = 1;
X_train = np.zeros((x_train.shape[0]-m, x_train.shape[1], x_train.shape[2], m, x_train.shape[3]));
Y_train = np.zeros((x_train.shape[0]-m, x_train.shape[1], x_train.shape[2], x_train.shape[3]));

for i in range(x_train.shape[0]-m):
    for j in range(m):
        X_train[i,:,:,j,:] = x_train[i+j]
    Y_train[i] = x_train[i+m]

X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[4]*m])
X_train = X_train.astype(np.float32) / 255
Y_train = Y_train.astype(np.float32) / 255

In [ ]:
np.amax(X_train)

## Create models

In [ ]:
input_g = Input(shape = (18, 32, 3*m,), name='level0')
layerg0 = Conv2D(3, (3, 3), padding = 'same', activation='relu') (input_g)
layerg0 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg0)))
layerg0 = UpSampling2D()(layerg0)
aux_input_g0 = Input(shape = (36,64,3*m,),name='level1')
layerg0 = concatenate([aux_input_g0, layerg0],axis=-1)
#36
layerg1 = Conv2D(3, (3, 3), padding = 'same', activation='relu') (layerg0)
layerg1 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg1)))
layerg1 = UpSampling2D()(layerg1)
aux_input_g1 = Input(shape = (72,128,3*m,),name='level2')
layerg1 = concatenate([aux_input_g1, layerg1],axis=-1)
#72
layerg2 = Conv2D(3, (3, 3), padding = 'same', activation='relu') (layerg1)
layerg2 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg2)))
layerg2 = UpSampling2D()(layerg2)
aux_input_g2 = Input(shape = (144,256,3*m,),name='level3')
layerg2 = concatenate([aux_input_g2, layerg2],axis=-1)
#144
layerg3 = Conv2D(3, (3, 3), padding = 'same', activation='relu') (layerg2)
layerg3 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerg3)))
aux_input_g3 = Input(shape = (144,256,3*m,),name='level4')
output = concatenate([aux_input_g3, layerg3],axis=-1)

model_g = Model(inputs=[input_g, aux_input_g0, aux_input_g1, aux_input_g2, aux_input_g3], outputs=output)

In [ ]:
model_g.summary()

In [ ]:
#input_d = Input(shape = (144, 256, 3,))
input_d = Input(shape = (144, 256, 3*(m+1),))
layerd1 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (input_d)
layerd1 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd1)))

layerd2 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (layerd1)
layerd2 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd2)))

layerd3 = Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', activation='relu') (layerd2)
layerd3 = BatchNormalization(momentum=0.8)(Dropout(0.25)(LeakyReLU(alpha=0.2)(layerd3)))

layerd4 = Conv2D(3, (3, 3), padding = 'same', activation='relu') (layerd3)
layerd4 = Flatten()(layerd4)
output = Dense(1, activation='sigmoid')(layerd4)
model_d = Model(inputs=input_d, outputs=output);

In [ ]:
model_d.summary()

In [ ]:
out_g = model_g([input_g,aux_input_g0,aux_input_g1,aux_input_g2,aux_input_g3])
out = model_d(out_g)
model = Model(inputs=[input_g,aux_input_g0,aux_input_g1,aux_input_g2,aux_input_g3], outputs=out)

In [ ]:
idx = np.random.randint(0, X_train.shape[0], 8)
imgs = np.concatenate((X_train[idx], Y_train[idx]),axis=3);

## Train the model

In [ ]:
model_d.compile(loss='binary_crossentropy',
            optimizer='adam', metrics=['accuracy'])

model_g.compile(loss='binary_crossentropy',
            optimizer='adam')

model.compile(loss='binary_crossentropy',
            optimizer='adam', metrics=['accuracy'])

In [ ]:
np.concatenate((dataimgs, dataimgs)).shape

In [ ]:
print(x_train.shape)
plt.imshow(X_train[0,:,:,0:3])

In [ ]:
imz = gen_imgs[:,:,:,12:15]
print(imz.shape)

fig=plt.figure(figsize=(8, 8))
columns = 2
rows = 2
for i in range(1, columns*rows+1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(imz[i-1,:,:])
    #plt.imshow(imz[:,:,3*i-3:3*i])
plt.show()

## Results and prediction

In [ ]:
import numpy as np
import math

def psnr(img1, img2):
    mse = np.mean((img1-img2)**2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
print(psnr(Yhat_test[8,:,:,:],Y_test[8,:,:,:]))

In [ ]:
batch_size = 8
epochs = 401
half_batch = int(batch_size / 2)

for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------
    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = Y_train[idx]
    #Previous frames
    dataimgs = X_train[idx]
    # Sample noise and generate a half batch of new images
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = model_g.predict([noise, dataimgs])

    imgs = np.concatenate((dataimgs, imgs), 3);

    # Train the discriminator (real classified as ones and generated as zeros)
    d_loss_real = model_d.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = model_d.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample generator input
    noise = np.random.normal(0, 1, (batch_size, 100))
    idx1 = np.random.randint(0, X_train.shape[0], half_batch)
    dataimgs1 = X_train[idx1]
    # Train the generator (wants discriminator to mistake images as real)
    g_loss = model.train_on_batch([noise, np.concatenate((dataimgs, dataimgs1))], np.ones((batch_size, 1)))

    # Plot the progress
    if epoch%100 == 0:
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

stopper = EarlyStopping(monitor='acc',
                              min_delta=0,
                              patience=0,
                              verbose=1, mode='auto')
model.fit(X_train, Y_train, 
          batch_size=8, epochs=10, callbacks=[stopper], verbose=1)

Yhat_test = model_d.predict(X_test)

Y_test.shape

import matplotlib.pyplot as plt
plt.imshow(Yhat_test[8,:,:,:])

batch_size = 8
epochs = 400
half_batch = int(batch_size / 2)

for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------
    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]

    # Sample noise and generate a half batch of new images
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = model_g.predict(noise)

    # Train the discriminator (real classified as ones and generated as zeros)
    d_loss_real = model_d.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = model_d.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample generator input
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Train the generator (wants discriminator to mistake images as real)
    g_loss = model.train_on_batch(noise, np.ones((batch_size, 1)))

    # Plot the progress
    if epoch%10 == 0:
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))